<a href="https://colab.research.google.com/github/medaminechafii/spaceship-titanic/blob/main/mynewproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [365]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [366]:
data = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [367]:
X

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre


In [368]:
X.drop(["PassengerId","Name","Age","Cabin"],axis=1,inplace=True)

In [369]:
X.head()

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,False,303.0,70.0,151.0,565.0,2.0


In [370]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

# seperating categorical and numerical categories

In [371]:
object_cols = [col for col in X.columns if X_train[col].dtype =="object"]
num_cols = [col for col in X.columns if X_train[col].dtype in ["int64","float64"]]

In [372]:
print(object_cols)

['HomePlanet', 'CryoSleep', 'Destination', 'VIP']


In [373]:
print(num_cols)

['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


# imputation

In [374]:
from sklearn.impute import SimpleImputer

In [375]:
numerical_tranformer = SimpleImputer(strategy = "constant")
X_num_imputed = pd.DataFrame(numerical_tranformer.fit_transform(X_train[num_cols]))
X_num_imputed.columns = num_cols
X_num_imputed.index = X_train.index
X_num_imputed

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,0.0,559.0,0.0,15238.0,2799.0
5971,0.0,20.0,1.0,696.0,0.0
464,1821.0,0.0,47.0,29.0,0.0
4475,185.0,0.0,476.0,1810.0,53.0
8469,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
4373,0.0,0.0,0.0,0.0,0.0
7891,0.0,0.0,6.0,0.0,733.0
4859,523.0,0.0,21.0,4.0,811.0
3264,0.0,0.0,0.0,0.0,0.0


In [376]:
X_train[num_cols] = X_num_imputed
X_train

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,Europa,False,55 Cancri e,False,0.0,559.0,0.0,15238.0,2799.0
5971,Earth,False,TRAPPIST-1e,False,0.0,20.0,1.0,696.0,0.0
464,Mars,False,TRAPPIST-1e,False,1821.0,0.0,47.0,29.0,0.0
4475,Earth,False,TRAPPIST-1e,False,185.0,0.0,476.0,1810.0,53.0
8469,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4373,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
7891,Earth,False,TRAPPIST-1e,False,0.0,0.0,6.0,0.0,733.0
4859,Mars,False,TRAPPIST-1e,False,523.0,0.0,21.0,4.0,811.0
3264,Earth,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0


In [377]:
categorical_transformer = SimpleImputer(strategy = "most_frequent")
X_cat_imputed = pd.DataFrame(categorical_transformer.fit_transform(X_train[object_cols]))
X_cat_imputed.columns = object_cols
X_cat_imputed.index = X_train.index
X_train[object_cols] = X_cat_imputed
X_train

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,Europa,False,55 Cancri e,False,0.0,559.0,0.0,15238.0,2799.0
5971,Earth,False,TRAPPIST-1e,False,0.0,20.0,1.0,696.0,0.0
464,Mars,False,TRAPPIST-1e,False,1821.0,0.0,47.0,29.0,0.0
4475,Earth,False,TRAPPIST-1e,False,185.0,0.0,476.0,1810.0,53.0
8469,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4373,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
7891,Earth,False,TRAPPIST-1e,False,0.0,0.0,6.0,0.0,733.0
4859,Mars,False,TRAPPIST-1e,False,523.0,0.0,21.0,4.0,811.0
3264,Earth,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0


In [378]:
na_columns = X_train.isnull().sum()
print(na_columns)

HomePlanet      0
CryoSleep       0
Destination     0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64


# One Hot Encoding

In [379]:
from sklearn.preprocessing import OneHotEncoder
OH = OneHotEncoder(handle_unknown = "ignore",sparse_output = False)
X_cat_OH = pd.DataFrame(OH.fit_transform(X_train[object_cols]))
X_cat_OH.index = X_train.index
X_train.drop(object_cols,axis = 1,inplace = True)
X_train_OH = pd.concat([X_train,X_cat_OH],axis = 1)
X_train_OH.columns = X_train_OH.columns.astype(str)
X_train_OH


,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
4278,0.0,559.0,0.0,15238.0,2799.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5971,0.0,20.0,1.0,696.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
464,1821.0,0.0,47.0,29.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4475,185.0,0.0,476.0,1810.0,53.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
8469,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
7891,0.0,0.0,6.0,0.0,733.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4859,523.0,0.0,21.0,4.0,811.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3264,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


# encoding the dependant variable

In [380]:
y_train

,Transported
4278,False
5971,False
464,False
4475,False
8469,True
...,...
4373,True
7891,False
4859,False
3264,False


In [381]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_OH = pd.DataFrame(le.fit_transform(y_train))
y_train_OH.index = y_train.index
y_train_OH

,0
4278,0
5971,0
464,0
4475,0
8469,1
...,...
4373,1
7891,0
4859,0
3264,0


# model creation

In [382]:
from sklearn.svm import SVC
model = SVC(kernel = 'rbf', random_state = 0)
model.fit(X_train_OH,y_train_OH.values.ravel())

RandomForestClassifier(n_estimators=200, random_state=0)

# preparing the test data

In [383]:
X_test_num_imputed = pd.DataFrame(numerical_tranformer.transform(X_test[num_cols]))
X_test_num_imputed.columns = num_cols
X_test_num_imputed.index = X_test.index
X_test[num_cols] = X_test_num_imputed

In [384]:
X_test_cat_imputed = pd.DataFrame(categorical_transformer.transform(X_test[object_cols]))
X_test_cat_imputed.columns = object_cols
X_test_cat_imputed.index = X_test.index
X_test[object_cols] = X_test_cat_imputed

In [385]:
X_test_cat_OH = pd.DataFrame(OH.transform(X_test[object_cols]))
X_test_cat_OH.index = X_test.index
X_test.drop(object_cols,axis = 1,inplace = True)
X_test_OH = pd.concat([X_test,X_test_cat_OH],axis = 1)
X_test_OH.columns = X_test_OH.columns.astype(str)

In [386]:
X_test_OH

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
3601,0.0,0.0,623.0,40.0,209.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
6057,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2797,517.0,1.0,301.0,60.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
7110,653.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
8579,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,0.0,760.0,234.0,2761.0,81.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3780,7.0,61.0,110.0,708.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6046,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
170,0.0,0.0,8.0,1072.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [387]:
y_test_OH = le.transform(y_test)

In [388]:
y_test_OH

array([1, 1, 0, ..., 1, 0, 1])

In [389]:
y_pred = model.predict(X_test_OH)

In [390]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test_OH, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[648 215]
 [166 710]]


0.7809085681426107

In [391]:
X_valid = data_test.copy()
the_ID = X_valid["PassengerId"].copy()
X_valid.drop(["PassengerId","Name","Age","Cabin"],axis=1,inplace=True)

In [392]:
X_valid_num_imputed = pd.DataFrame(numerical_tranformer.transform(X_valid[num_cols]))
X_valid_num_imputed.columns = num_cols
X_valid_num_imputed.index = X_valid.index
X_valid[num_cols] = X_valid_num_imputed

In [393]:
X_valid_cat_imputed = pd.DataFrame(categorical_transformer.transform(X_valid[object_cols]))
X_valid_cat_imputed.columns = object_cols
X_valid_cat_imputed.index = X_valid.index
X_valid[object_cols] = X_valid_cat_imputed

In [394]:
X_valid_cat_OH = pd.DataFrame(OH.transform(X_valid[object_cols]))
X_valid_cat_OH.index = X_valid.index
X_valid.drop(object_cols,axis = 1,inplace = True)
X_valid_OH = pd.concat([X_valid,X_valid_cat_OH],axis = 1)
X_valid_OH.columns = X_valid_OH.columns.astype(str)

In [395]:
y_valid_res = model.predict(X_valid_OH)

In [396]:
y_valid_res = y_valid_res==1
y_pred = pd.DataFrame({"PassengerId":the_ID,"Transported":y_valid_res})
y_pred.to_csv("submission.csv",index=False)